# Crashes Near Point

Find the crash incident_id for a specific crash given its location and time. 

In [1]:
import folium
import datetime
import pandas as pd
from folium.plugins import HeatMap

import os
os.chdir('..')
from scripts.crash_data_analysis import CrashDataAnalysis

In [2]:
cda = CrashDataAnalysis()
# df = cda.crash_dataframe(verbose=True)

scooter rider injured in hit and run 

https://www.denver7.com/news/local-news/denver-woman-recovering-from-serious-injuries-after-hit-and-run-while-riding-scooter

incident_id = 2023182255

In [3]:
# University Blvd and E 3rd Ave (2022,12,14) incident_id = 2022639830
# lat_lon = [39.720934, -104.959436]

# Speer Blvd and W 11th Ave
# lat_lon = [39.73378086237446, -104.99382870794332]

# Sheridan and W 38th Ave
# lat_lon = [39.76932995840248, -105.05322879106377]

# 18th St and Wazee - 2023-01-07
# lat_lon = [39.753095972145644, -104.99765515593928]

# 17th Ave and Humboldt
# lat_lon = [39.74325582993406, -104.96969463423684]

# Evans and Holly
# lat_lon = [39.678456222120396, -104.92219658639499]

# Perry and 32nd
# lat_lon = [39.76207714280117, -105.03931855940714]

# W 23rd Ave and Julian
# lat_lon = [39.751187102730654, -105.03155988650708]

# 35th St and Welton
# lat_lon = [39.76762137425839, -104.97535848218475]

# 1776 Race St
lat_lon = [39.744484814811436, -104.96299454652576]

# 730 S Canosa Ct
# lat_lon = [39.70332702509137, -105.01894998129806]

# Sherman St house
# lat_lon = [39.721383, -104.984736]

# Broadway and Colfax - scooter rider injured in hit and run
# lat_lon = [39.74012825562107, -104.98743057571703]

# lat_lon = cda.geocode_location('W 14th Ave & Federal Blvd, Denver, CO')

In [4]:
# crash occurred within this distance of the given latitude and longitude
radius_miles = 0.25

query = f"""
select
*
, reported_date at time zone 'America/Denver' as reported_date_local
, ST_Distance(
    geom_denver, ST_Transform(ST_SetSRID(ST_MakePoint({lat_lon[1]},{lat_lon[0]}), 4326), 3502)
) as distance_feet

from crashes

where geo_lon is not null
and ST_Distance(
    geom_denver, ST_Transform(ST_SetSRID(ST_MakePoint({lat_lon[1]},{lat_lon[0]}), 4326), 3502)
    ) < {radius_miles * 5280}

order by reported_date
"""
# and extract(year from reported_date) >= 2022

nearby = pd.read_sql(query, cda.conn)

In [5]:
nearby.groupby(nearby.reported_date.dt.year).agg(
    crashes=('incident_id', 'count')
    , fatalities=('fatality', 'sum')
    , sbi=('sbi', 'sum')
    , pedestrian_ind=('pedestrian_ind', 'sum')
    , bicycle_ind=('bicycle_ind', 'sum')
).astype(int)

,crashes,fatalities,sbi,pedestrian_ind,bicycle_ind
reported_date,,,,,
2013,92,0,0,2,0
2014,65,0,0,1,0
2015,98,0,0,0,4
2016,110,0,2,0,3
2017,103,0,1,1,1
2018,104,0,0,3,0
2019,102,0,1,1,2
2020,63,0,0,2,1
2021,53,0,2,2,1


In [6]:
nearby['all_time'] = 'all time'
nearby.groupby('all_time').agg(
    crashes=('incident_id', 'count')
    , fatalities=('fatality', 'sum')
    , sbi=('sbi', 'sum')
    , pedestrian_ind=('pedestrian_ind', 'sum')
    , bicycle_ind=('bicycle_ind', 'sum')
).astype(int)

,crashes,fatalities,sbi,pedestrian_ind,bicycle_ind
all_time,,,,,
all time,871,0,6,12,13


In [7]:
display_cols = [
    'reported_date_local'
    , 'top_traffic_accident_offense'
    , 'incident_address_corrected'
    , 'bicycle_ind'
    , 'pedestrian_ind'
    , 'sbi'
    , 'fatality'
    , 'distance_feet'
    , 'incident_id'
]
# nearby.to_clipboard()
# nearby[display_cols]

In [8]:
nearby[display_cols] #.tail(5)

,reported_date_local,top_traffic_accident_offense,incident_address_corrected,bicycle_ind,pedestrian_ind,sbi,fatality,distance_feet,incident_id
0,2013-01-03 09:04:00.000000,TRAF - ACCIDENT,E 17TH AVE / N YORK ST,0.0,0.0,False,False,1007.705554,20133651
1,2013-01-03 15:45:00.000000,TRAF - ACCIDENT,E 17TH AVE / N WILLIAMS ST,0.0,0.0,False,False,957.925309,20134223
2,2013-01-03 17:02:00.000000,TRAF - ACCIDENT,E 16TH AVE / N HIGH ST,0.0,0.0,False,False,1151.364351,20134287
3,2013-01-04 11:49:00.000000,TRAF - ACCIDENT,E 18TH AVE / N GILPIN ST,0.0,0.0,False,False,1191.772715,20135435
4,2013-01-06 14:03:00.000000,TRAF - ACCIDENT,2000 BLOCK E 16TH AVE,0.0,0.0,False,False,1067.653483,20139030
...,...,...,...,...,...,...,...,...,...
866,2023-03-16 15:15:00.000000,TRAF - ACCIDENT,N JOSEPHINE ST / E 17TH AVE,0.0,0.0,False,False,1137.464388,2023137860
867,2023-03-30 16:41:00.000001,TRAF - ACCIDENT,N JOSEPHINE ST / E 17TH AVE,0.0,0.0,False,False,1137.464388,2023164138
868,2023-04-17 08:28:00.000000,TRAF - ACCIDENT,E 17TH AVE / N HIGH ST,0.0,0.0,False,False,676.370556,2023197845
869,2023-04-19 10:17:00.000000,TRAF - ACCIDENT,1700 BLOCK N VINE ST,0.0,0.0,False,False,481.601833,2023202077


In [9]:
this_map = folium.Map(prefer_canvas=True)
HeatMap(nearby[['geo_lat', 'geo_lon']]).add_to(this_map)
this_map.fit_bounds(this_map.get_bounds())
this_map

In [10]:
cda.conn.dispose()